## Bribes research

In [18]:
import math

import pandas as pd
import requests
from IPython.core.display_functions import display

from notebooks.bribes_research.constants import BRIBES_URL
from notebooks.bribes_research.constants import DASHBOARD_URL
from notebooks.bribes_research.constants import PROTOCOL_AURA
from notebooks.bribes_research.constants import PROTOCOL_CVX
from notebooks.bribes_research.constants import TABLE_COLUMNS
from notebooks.bribes_research.helpers import calculate_metrics
from notebooks.bribes_research.helpers import fetch_token_prices

pd.set_option('display.max_rows', 500)
BADGER_BRIBE = 42000
STEP = 2000
BADGER_LOW_BOUND = 2000
BADGER_UPPER_BOUND = BADGER_BRIBE
# HH and Votium take 4% on all bribes
FEE = 0.04  # in %

REBALANCING_STEP = 0.01  # in %

table_rows = []
token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()
total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']

cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000066; color: white;'
}
for bribe in range(BADGER_LOW_BOUND, BADGER_UPPER_BOUND, STEP):
    badger_bribe_in_dollar = bribe * badger_price
    fees = badger_bribe_in_dollar * FEE
    badger_bribe_in_dollar_after_fees = badger_bribe_in_dollar - fees
    cvx_dilution_50, cvx_in_out_50 = calculate_metrics(badger_bribe_in_dollar_after_fees / 2, total_bribes_cvx,
                                                       dollar_spent_on_bribes_cvx)
    aura_dilution_50, aura_in_out_50 = calculate_metrics(badger_bribe_in_dollar_after_fees / 2, total_bribes_aura,
                                                         dollar_spent_on_bribes_aura)

    # Rebalance here
    cvx_in_out_floating = cvx_in_out_50
    aura_in_out_floating = aura_in_out_50

    cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar_after_fees / 2
    cvx_dilution = None
    aura_dilution = None
    while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
        # Do not allow negative numbers
        cvx_bribe_portion_percent = round(cvx_bribe_portion / badger_bribe_in_dollar_after_fees * 100, 2)
        aura_bribe_portion_percent = round(aura_bribe_portion / badger_bribe_in_dollar_after_fees * 100, 2)
        if math.isclose(cvx_bribe_portion_percent, 0, abs_tol=1) or math.isclose(aura_bribe_portion_percent, 0, abs_tol=1):
            break
        if cvx_in_out_floating > aura_in_out_floating:
            # Take bribe portion from aura and give it to cvx
            bribe_stolen = aura_bribe_portion * REBALANCING_STEP
            aura_bribe_portion -= bribe_stolen
            cvx_bribe_portion += bribe_stolen
        elif aura_in_out_floating > cvx_in_out_floating:
            # Take bribe portion from cvx and give it to aura
            bribe_stolen = aura_bribe_portion * REBALANCING_STEP
            cvx_bribe_portion -= bribe_stolen
            aura_bribe_portion += bribe_stolen
        if cvx_bribe_portion <= 0 or aura_bribe_portion <= 0:
            break
        cvx_dilution, cvx_in_out_floating = calculate_metrics(cvx_bribe_portion, total_bribes_cvx,
                                                              dollar_spent_on_bribes_cvx)
        aura_dilution, aura_in_out_floating = calculate_metrics(aura_bribe_portion, total_bribes_aura,
                                                                dollar_spent_on_bribes_aura)

    table_rows.append(
        [PROTOCOL_CVX, badger_bribe_in_dollar_after_fees, fees, round(cvx_bribe_portion / badger_bribe_in_dollar_after_fees * 100, 2),
         cvx_bribe_portion, cvx_dilution, cvx_in_out_floating]
    )
    table_rows.append(
        [PROTOCOL_AURA, badger_bribe_in_dollar_after_fees, fees, round(aura_bribe_portion / badger_bribe_in_dollar_after_fees * 100, 2),
         aura_bribe_portion, aura_dilution, aura_in_out_floating]
    )
dataframe = pd.DataFrame(data=table_rows, columns=TABLE_COLUMNS)
display(dataframe)

,Protocol,BADGER total bribe in $,Fees,Bribes Portion %,Bribes Portion,Dilution,$I/O
0,CVX,8179.2,340.8,0.66,53.596563,0.001988,149.245102
1,AURA,8179.2,340.8,99.34,8125.603437,2.610087,245.097946
2,CVX,16358.4,681.6,0.66,107.193126,0.003977,149.242135
3,AURA,16358.4,681.6,99.34,16251.206874,5.220175,238.529226
4,CVX,24537.6,1022.4,0.66,160.789689,0.005965,149.239167
5,AURA,24537.6,1022.4,99.34,24376.810311,7.830262,231.960505
6,CVX,32716.8,1363.2,0.66,214.386252,0.007954,149.236199
7,AURA,32716.8,1363.2,99.34,32502.413748,10.440350,225.391785
8,CVX,40896.0,1704.0,0.66,267.982815,0.009942,149.233232
9,AURA,40896.0,1704.0,99.34,40628.017185,13.050437,218.823065
